In [29]:
import qiskit
from qiskit import IBMQ

import numpy as np
from qiskit.opflow import MatrixOp
from qiskit.utils import QuantumInstance
from qiskit.opflow import PauliExpectation
from qiskit.opflow import CircuitSampler
from qiskit.circuit import QuantumCircuit, QuantumRegister, Parameter, ParameterVector, ParameterExpression
from qiskit.opflow import Z, X, I, StateFn, CircuitStateFn, SummedOp

from itertools import product

In [30]:
# IBMQ.save_account("TOKEN")
IBMQ.load_account() # Load account from disk
IBMQ.providers()    # List all available providers

ibmqfactory.load_account:WARNING:2021-11-22 15:14:33,718: Credentials are already in use. The existing account in the session will be replaced.


[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>,
 <AccountProvider for IBMQ(hub='ibm-q-research-2', group='epfl-4', project='main')>]

In [31]:
provider = IBMQ.get_provider(hub='ibm-q-research-2')
backend = provider.get_backend('ibmq_guadalupe')

In [9]:
def circuit(string = "0000", n_layers = 4, n_qubits = 4):
    params = ParameterVector('theta', length=n_layers*n_qubits)
    it = iter(params)
    C = QuantumCircuit(n_qubits)
    C.initialize(string)

    for j in range(n_layers):
        for i in range(n_qubits):
            C.rx(next(it), i)

        for i in range(n_layers-1):
            C.cx(i, i+1)

        C.barrier()
    return C, params

In [27]:
diag = np.arange(-1, 15)
H = MatrixOp(np.diag(diag)).to_pauli_op()

q_instance = QuantumInstance(backend=backend)#, shots=8192)

n_qubits = 4
n_layers = 2


def loss_fct(value):
    out = 0
    for i, string in enumerate([''.join(p) for p in product('10', repeat=n_qubits)]):
        circ1, _ = circuit(string = string, n_qubits = n_qubits, n_layers = n_layers)
        circ1 = circ1.decompose()
        circ1_bound = circ1.bind_parameters(value)
        psi = CircuitStateFn(primitive=circ1_bound, coeff=1.)
        measurable_expression = StateFn(H, is_measurement=True).compose(psi) 
        expectation = PauliExpectation().convert(measurable_expression)  
        sampler = CircuitSampler(q_instance).convert(expectation) 
        energy = sampler.eval().real
        out += (energy - diag[i])**2
    return out

In [32]:
value = np.random.rand(n_qubits*n_layers)
loss_fct(value)